In [2]:
# Step 1: Download and unzip dataset from Roboflow
!curl -L "https://app.roboflow.com/ds/t6xM6xE9bF?key=ym8EYKP9vQ" > roboflow.zip
!unzip roboflow.zip -d dataset && rm roboflow.zip




  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   904  100   904    0     0   2886      0 --:--:-- --:--:-- --:--:--  2897
100 17.3M  100 17.3M    0     0  16.2M      0  0:00:01  0:00:01 --:--:-- 81.8M
Archive:  roboflow.zip
replace dataset/README.dataset.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: dataset/README.dataset.txt  
  inflating: dataset/README.roboflow.txt  
  inflating: dataset/data.yaml       
 extracting: dataset/test/images/07-is-250-came-to-my-car-saw-a-big-crack-in-the-rear-v0-epzwgdws1iaa1_webp.rf.f8fa8c1e25c320516c8be752fc2c6fbe.jpg  
 extracting: dataset/test/images/073_jpg.rf.9fb7792655a0d1aeb80a19d88d2216c2.jpg  
 extracting: dataset/test/images/077_jpg.rf.27e0ff0931694f718ff9ce851b1a7ae9.jpg  
 extracting: dataset/test/images/1150_jpg.rf.e2a4e07bcca74f8c0a698a4181be1d18.jpg  
 extracting: dataset/test/images/116260-IMG_9532_JPG_jpg.rf.ca

In [3]:
import os

for root, dirs, files in os.walk("/content/dataset"):
    print(f"\n📁 Directory: {root}")
    for d in dirs:
        print(f" - 📂 Sub-folder: {d}")
    for f in files[:5]:  # show only first 5 files to keep it short
        print(f"   - 🖼️ File: {f}")



📁 Directory: /content/dataset
 - 📂 Sub-folder: test
 - 📂 Sub-folder: train
 - 📂 Sub-folder: valid
   - 🖼️ File: data.yaml
   - 🖼️ File: README.dataset.txt
   - 🖼️ File: README.roboflow.txt

📁 Directory: /content/dataset/test
 - 📂 Sub-folder: images
 - 📂 Sub-folder: labels

📁 Directory: /content/dataset/test/images
   - 🖼️ File: j1ize4m6cj3a1_webp.rf.ade4cbe88d88880275bf87febbc0b5b1.jpg
   - 🖼️ File: 073_jpg.rf.9fb7792655a0d1aeb80a19d88d2216c2.jpg
   - 🖼️ File: i-30-_30_11zon_jpg.rf.a3d1dce0c741e8a57d1e17863ccc153b.jpg
   - 🖼️ File: amaaagok4oa_1920_jpg.rf.6ee01b789b1f3cd1d0865a4e66f660ab.jpg
   - 🖼️ File: images-61-_jpg.rf.1752afb0f903c2288b27134bde8f0d69.jpg

📁 Directory: /content/dataset/test/labels
   - 🖼️ File: 272_6_jpg.rf.ef48bd636a977979813113759191e081.txt
   - 🖼️ File: car2_jpg.rf.563c9fc1c91304597eaada3e5148f137.txt
   - 🖼️ File: img30_jpg.rf.e4134e01f2d54d06f509c42fe2e0a14b.txt
   - 🖼️ File: IMG_0003-22-_JPG_jpg.rf.9e793ede25727926890729f14ccfffd4.txt
   - 🖼️ File: 073_jpg.

In [4]:
# Step 1: Import libraries
import os
import torch
import torchvision
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report

# Step 2: Define YOLO-style custom Dataset
class YOLOClassificationDataset(Dataset):
    def __init__(self, images_dir, labels_dir, transform=None):
        self.images_dir = images_dir
        self.labels_dir = labels_dir
        self.transform = transform
        self.image_files = sorted([
            f for f in os.listdir(images_dir)
            if f.endswith(('.jpg', '.jpeg', '.png'))
        ])

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_name = self.image_files[idx]
        img_path = os.path.join(self.images_dir, img_name)
        label_path = os.path.join(self.labels_dir, os.path.splitext(img_name)[0] + '.txt')

        # Load image
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        # Load first class_id from label file (assume single label per image)
        with open(label_path, 'r') as f:
            class_id = int(f.readline().split()[0])

        return image, class_id

# Step 3: Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Step 4: Set paths for train and validation
train_images = '/content/dataset/train/images'
train_labels = '/content/dataset/train/labels'
val_images = '/content/dataset/valid/images'
val_labels = '/content/dataset/valid/labels'

# Step 5: Create Datasets and Dataloaders
train_dataset = YOLOClassificationDataset(train_images, train_labels, transform=transform)
val_dataset = YOLOClassificationDataset(val_images, val_labels, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Step 6: Prepare ResNet-50 Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(set(label for _, label in train_dataset))  # infer classes

model = models.resnet50(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Step 7: Train the model
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss:.4f}")

# Step 8: Evaluate the model
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

print("\nClassification Report:\n")
print(classification_report(all_labels, all_preds))


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 103MB/s]


Epoch [1/5], Loss: 18.8070
Epoch [2/5], Loss: 7.6486
Epoch [3/5], Loss: 2.3688
Epoch [4/5], Loss: 0.5362
Epoch [5/5], Loss: 0.1915

Classification Report:

              precision    recall  f1-score   support

           0       0.44      0.80      0.57         5
           1       0.67      0.67      0.67         9
           2       1.00      1.00      1.00        13
           3       0.57      0.80      0.67         5
           4       0.93      0.93      0.93        14
           5       0.82      1.00      0.90         9
           6       0.83      0.36      0.50        14

    accuracy                           0.78        69
   macro avg       0.75      0.79      0.75        69
weighted avg       0.81      0.78      0.77        69



In [7]:
# Define class label names
class_names = ['Crack', 'Dent', 'PDR-Dent', 'Paint-Crack', 'Paint-fading', 'Rust', 'Scratch']

# Print classification report with labels
print("\nClassification Report:\n")
print(classification_report(all_labels, all_preds, target_names=class_names))




Classification Report:

              precision    recall  f1-score   support

       Crack       0.44      0.80      0.57         5
        Dent       0.67      0.67      0.67         9
    PDR-Dent       1.00      1.00      1.00        13
 Paint-Crack       0.57      0.80      0.67         5
Paint-fading       0.93      0.93      0.93        14
        Rust       0.82      1.00      0.90         9
     Scratch       0.83      0.36      0.50        14

    accuracy                           0.78        69
   macro avg       0.75      0.79      0.75        69
weighted avg       0.81      0.78      0.77        69



In [9]:
# Step 6: Prepare ResNet-18 Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_classes = len(set(label for _, label in train_dataset))  # infer classes

model = models.resnet18(pretrained=True)  # ⬅️ Changed to resnet18
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# Step 7: Train the model
epochs = 5
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss:.4f}")

# Step 8: Evaluate the model
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Optional: Add class names for labeling
class_names = ['Crack', 'Dent', 'PDR-Dent', 'Paint-Crack', 'Paint-fading', 'Rust', 'Scratch']

print("\nClassification Report:\n")
print(classification_report(all_labels, all_preds, target_names=class_names))


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 76.5MB/s]


Epoch [1/5], Loss: 19.0943
Epoch [2/5], Loss: 6.7770
Epoch [3/5], Loss: 2.6739
Epoch [4/5], Loss: 1.1452
Epoch [5/5], Loss: 0.5570

Classification Report:

              precision    recall  f1-score   support

       Crack       0.45      1.00      0.62         5
        Dent       0.67      0.67      0.67         9
    PDR-Dent       1.00      1.00      1.00        13
 Paint-Crack       0.60      0.60      0.60         5
Paint-fading       0.92      0.79      0.85        14
        Rust       0.89      0.89      0.89         9
     Scratch       0.60      0.43      0.50        14

    accuracy                           0.75        69
   macro avg       0.73      0.77      0.73        69
weighted avg       0.78      0.75      0.75        69

